# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Eberhardt  ->  J. Eberhardt  |  ['J. Eberhardt']
T. Henning  ->  T. Henning  |  ['T. Henning']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
Arxiv has 68 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2412.02769


extracting tarball to tmp_2412.02769...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2412.02769/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'system_parameters_planet_v2' from 'tmp_2412.02769/system_parameters_planet_v2.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'system_parameters_v2' from 'tmp_2412.02769/system_parameters_v2.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/

...

Found 138 bibliographic references in tmp_2412.02846/ms.bbl.
Retrieving document from  https://arxiv.org/e-print/2412.03063
extracting tarball to tmp_2412.03063... done.
Retrieving document from  https://arxiv.org/e-print/2412.03411


extracting tarball to tmp_2412.03411...

 done.


E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']


Found 75 bibliographic references in tmp_2412.03411/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.02769-b31b1b.svg)](https://arxiv.org/abs/2412.02769) | **Discovery and Characterization of an Eccentric, Warm Saturn Transiting the Solar Analog TOI-4994**  |
|| R. R. Martinez, et al. -- incl., <mark>J. Eberhardt</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2024-12-05*|
|*Comments*| *17 pages, 10 figures. Accepted to AJ*|
|**Abstract**|            We present the detection and characterization of TOI-4994b (TIC 277128619b), a warm Saturn-sized planet discovered by the NASA Transiting Exoplanet Survey Satellite (TESS). TOI-4994b transits a G-type star (V = 12.6 mag) with a mass, radius, and effective temperature of $M_{\star} =1.005^{+0.064}_{-0.061} M_{\odot}$, $R_{\star} = 1.055^{+0.040}_{-0.037} R_{\odot}$, and $T_{\rm eff} = 5640 \pm 110$ K. We obtained follow-up ground-based photometry from the Las Cumbres Observatory (LCO) and the Antarctic Search for Transiting ExoPlanets (ASTEP) telescopes, and we confirmed the planetary nature of TOI-4994b with multiple radial velocity observations from the PFS, CHIRON, HARPS, FEROS, and CORALIE instruments. From a global fit to the photometry and radial velocities, we determine that TOI-4994b is in a 21.5-day, eccentric orbit ($e = 0.32 \pm 0.04$) and has a mass of $M_{P}= 0.280^{+0.037}_{-0.034} M_{J}$, a radius of $R_{P}= 0.762^{+0.030}_{-0.027}R_{J}$, and a Saturn-like bulk density of $\rho_{p} = 0.78^{+0.16}_{-0.14}$ $\rm g/cm^3$. We find that TOI-4994 is a potentially viable candidate for follow-up stellar obliquity measurements. TOI-4994b joins the small sample of warm Saturn analogs and thus sheds light on our understanding of these rare and unique worlds.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.02846-b31b1b.svg)](https://arxiv.org/abs/2412.02846) | **Environmental Evidence for Overly Massive Black Holes in Low Mass Galaxies and a Black Hole - Halo Mass Relation at $z \sim 5$**  |
|| J. Matthee, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2024-12-05*|
|*Comments*| *19 pages, 15 figures. Submitted to ApJ. Main results in Figures 12, 13, 14. Comments welcome*|
|**Abstract**|            JWST observations have unveiled faint active galactic nuclei (AGN) at high-redshift that provide insights on the formation of supermassive black holes (SMBHs) and their coevolution with galaxies. However, disentangling stellar from AGN light in these sources is challenging. Here, we use an empirical approach to infer the average stellar mass of 6 faint broad line (BL) Halpha emitters at z = 4 - 5 with BH masses ~ 6 (4 - 15)x10^6 Msun, with a method independent of their spectral energy distribution (SED). We use the deep JWST/NIRcam grism survey ALT to measure the over-densities around BL-Halpha emitters and around a spectroscopic reference sample of ~300 galaxies. In our reference sample, we find that Mpc-scale over-density correlates with stellar mass, while pair counts are flat below ~50 kpc due to satellites. Their large-scale environments suggest that BL-Halpha emitters are hosted by galaxies with stellar masses ~5x10^7 Msun, ~40 times lower than those inferred from galaxy-only SED fits. Adding measurements around more luminous z~6 AGNs, we find tentative correlations between line width, BH mass and the over-density, suggestive of a steep BH to halo mass relation. The main implications are (1) when BH masses are taken at face value, we confirm extremely high BH to stellar mass ratios of ~10 %, (2) the low stellar mass galaxies hosting growing SMBHs are in tension with typical hydrodynamical simulations, except those without feedback, (3) a 1 % duty cycle implied by the host mass hints at super-Eddington accretion, which may imply over-estimated SMBH masses, (4) the masses are at odds with a high stellar density interpretation of the line broadening, (5) our results imply a diversity of galaxy masses, environments and SEDs among AGN samples, depending on their luminosity.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.03411-b31b1b.svg)](https://arxiv.org/abs/2412.03411) | **A dark, bare rock for TOI-1685 b from a JWST NIRSpec G395H phase curve**  |
|| R. Luque, et al. -- incl., <mark>E.-M. Ahrer</mark> |
|*Appeared on*| *2024-12-05*|
|*Comments*| *26 pages, 19 figures, 8 tables. Submitted to AAS Journals*|
|**Abstract**|            We report JWST NIRSpec/G395H observations of TOI-1685 b, a hot rocky super-Earth orbiting an M2.5V star, during a full orbit. We obtain transmission and emission spectra of the planet and characterize the properties of the phase curve, including its amplitude and offset. The transmission spectrum rules out clear H$_2$-dominated atmospheres, while secondary atmospheres (made of water, methane, or carbon dioxide) cannot be statistically distinguished from a flat line. The emission spectrum is featureless and consistent with a blackbody-like brightness temperature, helping rule out thick atmospheres with high mean molecular weight. Collecting all evidence, the properties of TOI-1685 b are consistent with a blackbody with no heat redistribution and a low albedo, with a dayside brightness temperature 0.98$\pm$0.07 times that of a perfect blackbody in the NIRSpec NRS2 wavelength range (3.823-5.172 um). Our results add to the growing number of seemingly airless M-star rocky planets, thus constraining the location of the "Cosmic Shoreline". Three independent data reductions have been carried out, all showing a high-amplitude correlated noise component in the white and spectroscopic light curves. The correlated noise properties are different between the NRS1 and NRS2 detectors - importantly the timescales of the strongest components (4.5 hours and 2.5 hours, respectively) - suggesting the noise is from instrumental rather than astrophysical origins. We encourage the community to look into the systematics of NIRSpec for long time-series observations.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.03063-b31b1b.svg)](https://arxiv.org/abs/2412.03063) | **The FAST Galactic Plane Pulsar Snapshot Survey: VII. Six millisecond pulsars in compact orbits with massive white dwarf companions**  |
|| Z. L. Yang, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2024-12-05*|
|*Comments*| *12 pages, 6 figures, submitted to RAA*|
|**Abstract**|            Binary millisecond pulsars with a massive white dwarf (WD) companion are intermediate-mass binary pulsars (IMBPs). They are formed via the Case BB Roche-lobe overflow (RLO) evolution channel if they are in compact orbits with an orbital period of less than 1 day. They are fairly rare in the known pulsar population, only five such IMBPs have been discovered before, and one of them is in a globular cluster. Here we report six IMBPs in a compact orbit, PSRs J0416+5201, J0520+3722, J1919+1341, J1943+2210, J1947+2304 and J2023+2853, discovered during the Galactic Plane Pulsar Snapshot (GPPS) survey by using the Five-hundred-meter Aperture Spherical radio Telescope (FAST), doubling the number of such IMBPs due to the high survey sensitivity in the short survey time of 5 minutes. Follow-up timing observations show that they all have either a CO WD or an ONeMg WD companion with a mass greater than about 0.8~M$_\odot$ in a very circular orbit with an eccentricity in the order of $\lesssim10^{-5}$. PSR J0416+5201 should be an ONeMg WD companion with a remarkable minimum mass of 1.28 M$_\odot$. These massive white dwarf companions lead to a detectable Shapiro delay for PSRs J0416+5201, J0520+3722, J1943+2210, and J2023+2853, indicating that their orbits are highly inclined. From the measurement of the Shapiro delay, the pulsar mass of J1943+2210 was constrained to be 1.84$^{\,+0.11}_{-0.09}$~M$_\odot$, and that of PSR J2023+2853 to be 1.28$^{\,+0.06}_{-0.05}$~M$_\odot$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2412.02769/./Fig2.png', 'tmp_2412.02769/./Fig4.png', 'tmp_2412.02769/./Fig1.png']
copying  tmp_2412.02769/./Fig2.png to _build/html/
copying  tmp_2412.02769/./Fig4.png to _build/html/
copying  tmp_2412.02769/./Fig1.png to _build/html/
exported in  _build/html/2412.02769.md
    + _build/html/tmp_2412.02769/./Fig2.png
    + _build/html/tmp_2412.02769/./Fig4.png
    + _build/html/tmp_2412.02769/./Fig1.png
found figures ['tmp_2412.02846/./figs/MSTAR_MBH.png', 'tmp_2412.02846/./figs/MSTAR_MBH_sims.png', 'tmp_2412.02846/./figs/LHA_VFWHM.png']
copying  tmp_2412.02846/./figs/MSTAR_MBH.png to _build/html/
copying  tmp_2412.02846/./figs/MSTAR_MBH_sims.png to _build/html/
copying  tmp_2412.02846/./figs/LHA_VFWHM.png to _build/html/
exported in  _build/html/2412.02846.md
    + _build/html/tmp_2412.02846/./figs/MSTAR_MBH.png
    + _build/html/tmp_2412.02846/./figs/MSTAR_MBH_sims.png
    + _build/html/tmp_2412.02846/./figs/LHA_VFWHM.png
found figures ['tmp_2412.03411/./figures/pa

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand{\bjdtdb}{\ensuremath{\rm{BJD_{TDB}}}}$
$\newcommand{\feh}{\ensuremath{\left[{\rm Fe}/{\rm H}\right]}}$
$\newcommand{\mh}{\ensuremath{\left[{\rm m}/{\rm H}\right]}}$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}\xspace}$
$\newcommand{\logg}{\ensuremath{\log g}}$
$\newcommand{\ecosw}{\ensuremath{e\cos{\omega_*}}}$
$\newcommand{\esinw}{\ensuremath{e\sin{\omega_*}}}$
$\newcommand{\msun}{\ensuremath{ M_\Sun}}$
$\newcommand{\rsun}{\ensuremath{ R_\Sun}}$
$\newcommand{\lsun}{\ensuremath{ L_\Sun}}$
$\newcommand{\mj}{\ensuremath{ M_{\rm J}}}$
$\newcommand{\rj}{\ensuremath{ R_{\rm J}}}$
$\newcommand{\rearth}{\ensuremath{ R_{\rm \Earth}}\xspace}$
$\newcommand{\mearth}{\ensuremath{ M_{\rm \Earth}}\xspace}$
$\newcommand{\re}{\ensuremath{ R_{\rm \Earth}}\xspace}$
$\newcommand{\me}{\ensuremath{ M_{\rm \Earth}}\xspace}$
$\newcommand{\fave}{\langle F \rangle}$
$\newcommand{\fluxcgs}{10^9 erg s^{-1} cm^{-2}}$
$\newcommand{\newln}{\\&\quad\quad }$
$\newcommand{\Kepler}{{\it Kepler}}$
$\newcommand{\Ktwo}{{\it K2}}$
$\newcommand{\loggstar}{\ensuremath{\log{g_\star}}}$
$\newcommand{\vsini}{\ensuremath{v\sin{i_*}}}$
$\newcommand{\kms}{ km s^{-1}}$
$\newcommand{\ms}{ m s^{-1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\cfa}{Center for Astrophysics \textbar  Harvard \& Smithsonian, 60 Garden St, Cambridge, MA 02138, USA}$
$\newcommand{\msu}{Center for Data Intensive and Time Domain Astronomy, Department of Physics and Astronomy, Michigan State University, East$
$Lansing, MI 48824, USA}$
$\newcommand{\umich}{Astronomy Department, University of Michigan, 1085 S University Avenue, Ann Arbor, MI 48109, USA}$
$\newcommand{\utaustin}{Department of Astronomy, The University of Texas at Austin, Austin, TX 78712, USA}$
$\newcommand{\MIT}{Department of Physics and Kavli Institute for Astrophysics and Space Research, Massachusetts Institute of Technology, Cambridge, MA 02139, USA}$
$\newcommand{\MITEPS}{Department of Earth, Atmospheric and Planetary Sciences, Massachusetts Institute of Technology,  Cambridge,  MA 02139, USA}$
$\newcommand{\uflorida}{Department of Astronomy, University of Florida, 211 Bryant Space Science Center, Gainesville, FL, 32611, USA}$
$\newcommand{\riverside}{Department of Earth and Planetary Sciences, University of California, Riverside, CA 92521, USA}$
$\newcommand{\usq}{Centre for Astrophysics, University of Southern Queensland, West Street, Toowoomba, QLD 4350, Australia}$
$\newcommand{\ames}{NASA Ames Research Center, Moffett Field, CA, 94035, USA}$
$\newcommand{\geneva}{Observatoire astronomique de l\' Université de Genève, 51 Chemin Pegasi, 1290 Versoix, Switzerland}$
$\newcommand{\uw}{Astronomy Department, University of Washington, Seattle, WA 98195 USA}$
$\newcommand{\warwick}{Department of Physics, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK}$
$\newcommand{\warwickceh}{Centre for Exoplanets and Habitability, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK}$
$\newcommand{\princeton}{Department of Astrophysical Sciences, Princeton University, 4 Ivy Lane, Princeton, NJ, 08544, USA}$
$\newcommand{\liege}{Space Sciences, Technologies and Astrophysics Research (STAR) Institute, Université de Liège, 19C Allée du 6 Ao\^ut, 4000 Liège, Belgium}$
$\newcommand{\vanderbilt}{Department of Physics and Astronomy, Vanderbilt University, Nashville, TN 37235, USA}$
$\newcommand{\fisk}{Department of Physics, Fisk University, 1000 17th Avenue North, Nashville, TN 37208, USA}$
$\newcommand{\columbia}{Department of Astronomy, Columbia University, 550 West 120th Street, New York, NY 10027, USA}$
$\newcommand{\toronto}{Dunlap Institute for Astronomy and Astrophysics, University of Toronto, Ontario M5S 3H4, Canada}$
$\newcommand{\unc}{Department of Physics and Astronomy, University of North Carolina at Chapel Hill, Chapel Hill, NC 27599, USA}$
$\newcommand{\iac}{Instituto de Astrofísica de Canarias (IAC), E-38205 La Laguna, Tenerife, Spain}$
$\newcommand{\lalaguna}{Departamento de Astrofísica, Universidad de La Laguna (ULL), E-38206 La Laguna, Tenerife, Spain}$
$\newcommand{\louisville}{Department of Physics and Astronomy, University of Louisville, Louisville, KY 40292, USA}$
$\newcommand{\aavso}{American Association of Variable Star Observers, 49 Bay State Road, Cambridge, MA 02138, USA}$
$\newcommand{\utokyo}{The University of Tokyo, 7-3-1 Hongo, Bunky\={o}, Tokyo 113-8654, Japan}$
$\newcommand{\naoj}{National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo 181-8588, Japan}$
$\newcommand{\jstpresto}{JST, PRESTO, 7-3-1 Hongo, Bunkyo-ku, Tokyo 113-0033, Japan}$
$\newcommand{\astrobiojapan}{Astrobiology Center, 2-21-1 Osawa, Mitaka, Tokyo 181-8588, Japan}$
$\newcommand{\ctio}{Cerro Tololo Inter-American Observatory, Casilla 603, La Serena, Chile}$
$\newcommand{\noirlab}{NOIRLab/Cerro Tololo Inter-American Observatory, Casilla 603, La Serena, Chile}$
$\newcommand{\nexsci}{Caltech IPAC -- NASA Exoplanet Science Institute 1200 E. California Ave, Pasadena, CA 91125, USA}$
$\newcommand{\ucsc}{Department of Astronomy and Astrophysics, University of$
$California, Santa Cruz, CA 95064, USA}$
$\newcommand{\gsfc}{Exoplanets and Stellar Astrophysics Laboratory, Code 667, NASA Goddard Space Flight Center, Greenbelt, MD 20771, USA}$
$\newcommand{\sgtinc}{SGT, Inc./NASA AMES Research Center, Mailstop 269-3, Bldg T35C, P.O. Box 1, Moffett Field, CA 94035, USA}$
$\newcommand{\chile}{Center of Astro-Engineering UC, Pontificia Universidad Católica de Chile, Av. Vicuña Mackenna 4860, 7820436 Macul, Santiago, Chile}$
$\newcommand{\Pontificia}{Facultad de Ingeniería y Ciencias, Universidad Adolfo Ibáñez, Av. Diagonal las Torres 2640, Peñalolén, Santiago, Chile}$
$\newcommand{\Millennium}{Millennium Institute for Astrophysics, Chile}$
$\newcommand{\maxplank}{Max-Planck-Institut für Astronomie, Königstuhl 17, Heidelberg 69117, Germany}$
$\newcommand{\utdallas}{Department of Physics, The University of Texas at Dallas, 800 West$
$Campbell Road, Richardson, TX 75080-3021 USA}$
$\newcommand{\MauryLewin}{Maury Lewin Astronomical Observatory, Glendora, CA 91741, USA}$
$\newcommand{\umbc}{University of Maryland, Baltimore County, 1000 Hilltop Circle, Baltimore, MD 21250, USA}$
$\newcommand{\osu}{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, OH 43210, USA}$
$\newcommand{\MITAA}{Department of Aeronautics and Astronautics, MIT, 77 Massachusetts Avenue, Cambridge, MA 02139, USA}$
$\newcommand{\openu}{School of Physical Sciences, The Open University, Milton Keynes MK7 6AA, UK}$
$\newcommand{\swarthmore}{Department of Physics and Astronomy, Swarthmore College, Swarthmore, PA 19081, USA}$
$\newcommand{\seti}{SETI Institute, Mountain View, CA 94043, USA}$
$\newcommand{\lehigh}{Department of Physics, Lehigh University, 16 Memorial Drive East, Bethlehem, PA 18015, USA}$
$\newcommand{\utah}{Department of Physics and Astronomy, University of Utah, 115 South 1400 East, Salt Lake City, UT 84112, USA}$
$\newcommand{\USNA}{Department of Physics, United States Naval Academy, 572C Holloway Rd., Annapolis, MD 21402, USA}$
$\newcommand{\DTM}{Department of Terrestrial Magnetism, Carnegie Institution for Science, 5241 Broad Branch Road, NW, Washington, DC 20015, USA}$
$\newcommand{\UPenn}{The University of Pennsylvania, Department of Physics and Astronomy, Philadelphia, PA, 19104, USA}$
$\newcommand{\montana}{Department of Physics and Astronomy, University of Montana, 32 Campus Drive, No. 1080, Missoula, MT 59812 USA}$
$\newcommand{\psu}{Department of Astronomy \& Astrophysics, The Pennsylvania State University, 525 Davey Lab, University Park, PA 16802, USA}$
$\newcommand{\psust}{Center for Exoplanets and Habitable Worlds, The Pennsylvania State University, 525 Davey Lab, University Park, PA 16802, USA}$
$\newcommand{\Kutztown}{Department of Physical Sciences, Kutztown University, Kutztown, PA 19530, USA}$
$\newcommand{\udel}{Department of Physics \& Astronomy, University of Delaware, Newark, DE 19716, USA}$
$\newcommand{\Westminster}{Department of Physics, Westminster College, New Wilmington, PA 16172}$
$\newcommand{\steward}{Department of Astronomy and Steward Observatory, University of Arizona, Tucson, AZ 85721, USA}$
$\newcommand{\saao}{South African Astronomical Observatory, PO Box 9, Observatory, 7935, Cape Town, South Africa}$
$\newcommand{\salt}{Southern African Large Telescope, PO Box 9, Observatory, 7935, Cape Town, South Africa}$
$\newcommand{\ssl}{Società Astronomica Lunae, Italy}$
$\newcommand{\spot}{Spot Observatory, Nashville, TN 37206, USA}$
$\newcommand{\txamGP}{George P. and Cynthia Woods Mitchell Institute for Fundamental Physics and Astronomy, Texas A\&M University, College Station, TX77843 USA}$
$\newcommand{\txam}{Department of Physics and Astronomy, Texas A\&M university, College Station, TX 77843 USA}$
$\newcommand{\wellesley}{Department of Astronomy, Wellesley College, Wellesley, MA 02481, USA}$
$\newcommand{\Wesleyan}{Astronomy Department and Van Vleck Observatory, Wesleyan University, Middletown, CT 06459, USA}$
$\newcommand{\inaf}{INAF -- Osservatorio Astronomico di Padova, Vicolo dell'Osservatorio 5, I-35122, Padova, Italy}$
$\newcommand{\byu}{Department of Physics and Astronomy, Brigham Young University, Provo, UT 84602, USA}$
$\newcommand{\Hazelwood}{Hazelwood Observatory, Churchill, Victoria, Australia}$
$\newcommand{\austinstate}{Department of Physics, Engineering and Astronomy, Stephen F. Austin State University, 1936 North St, Nacogdoches, TX 75962, USA}$
$\newcommand{\pest}{Perth Exoplanet Survey Telescope}$
$\newcommand{\Winer}{Winer Observatory, PO Box 797, Sonoita, AZ 85637, USA}$
$\newcommand{\icpo}{Ivan Curtis Private Observatory}$
$\newcommand{\elsauce}{El Sauce Observatory, Chile}$
$\newcommand{\crow}{Atalaia Group \& CROW Observatory, Portalegre, Portugal}$
$\newcommand{\dfus}{Dipartimento di Fisica "E.R.Caianiello", Università di Salerno, Via Giovanni Paolo II 132, Fisciano 84084, Italy}$
$\newcommand{\indfn}{Istituto Nazionale di Fisica Nucleare, Napoli, Italy}$
$\newcommand{\sotes}{Gabriel Murawski Private Observatory (SOTES)}$
$\newcommand{\lco}{Las Cumbres Observatory Global Telescope, 6740 Cortona Dr., Suite 102, Goleta, CA 93111, USA}$
$\newcommand{\ucsb}{Department of Physics, University of California, Santa Barbara, CA 93106-9530, USA}$
$\newcommand{\yale}{Department of Astronomy, Yale University, 52 Hillhouse Avenue, New Haven, CT 06511, USA}$
$\newcommand{\eso}{European Southern Observatory, Alonso de Córdova 3107, Vitacura, Casilla 19001, Santiago, Chile}$
$\newcommand{\stsci}{Space Telescope Science Institute, Baltimore, MD 21218, USA}$
$\newcommand{\keele}{Astrophysics Group, Keele University, Staffordshire ST5 5BG, UK}$
$\newcommand{\gsfcsellers}{GSFC Sellers Exoplanet Environments Collaboration, NASA Goddard Space Flight Center, Greenbelt, MD 20771 }$
$\newcommand{\usno}{U.S. Naval Observatory, Washington, DC 20392, USA}$
$\newcommand{\kansas}{Department of Physics and Astronomy, University of Kansas, 1251 Wescoe Hall Dr., Lawrence, KS 66045, USA}$
$\newcommand{\gmu}{George Mason University, 4400 University Drive MS 3F3, Fairfax, VA 22030, USA}$
$\newcommand{\unsw}{Exoplanetary Science at UNSW, School of Physics, UNSW Sydney, NSW 2052, Australia}$
$\newcommand{\sifa}{School of Physics, Sydney Institute for Astronomy (SIfA), The University of Sydney, NSW 2006, Australia}$
$\newcommand{\nanjing}{School of Astronomy and Space Science, Key Laboratory of Modern Astronomy and Astrophysics in Ministry of Education, Nanjing University, Nanjing 210046, Jiangsu, China}$
$\newcommand{\berkely}{Department of Astronomy, University of California Berkeley, Berkeley, CA 94720-3411, USA}$
$\newcommand{\bhicfa}{Black Hole Initiative at Harvard University, 20 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\Silesian}{Department of Electronics, Electronical Engineering and Microelectronics, Silesian University of Techhnology Akademicka 16, 44-100 Gliwice, Poland}$
$\newcommand{\Patashnick}{Patashnick Voorheesville Observatory, Voorheesville, NY 12186, USA}$
$\newcommand{\austincollege}{Physics Department, Austin College, 900 North Grand Avenue, Sherman TX 75090, USA}$
$\newcommand{\Tsinghua}{Department of Astronomy, Tsinghua University, Beijing 100084, China}$
$\newcommand{\Tsinghuaschool}{Tsinghua International School, Beijing 100084, China}$
$\newcommand{\chinaNAO}{National Astronomical Observatories, Chinese Academy of Sciences, 20A Datun Road, Chaoyang District, Beijing 100012, China}$
$\newcommand{\Tautenburg}{Th{ü}ringer Landessternwarte Tautenburg, Sternwarte 5, 07778 Tautenburg, Germany}$
$\newcommand{\brierfield}{Brierfield Observatory, New South Wales, Australia}$
$\newcommand{\Indiana}{Indiana University Department of Astronomy, SW319, 727 E 3rd Street, Bloomington, IN 47405 USA}$
$\newcommand{\wisconsin}{Department of Astronomy, University of Wisconsin-Madison, Madison, WI 53706, USA}$
$\newcommand{\protologic}{Proto-Logic Consulting LLC, Washington, DC 20009, USA}$
$\newcommand{\ASTRAVEO}{ASTRAVEO LLC, PO Box 1668, MA 01931}$
$\newcommand{\TJHS}{Thomas Jefferson High School, 6560 Braddock Rd, Alexandria, VA 22312 USA}$
$\newcommand{\ucatchile}{Instituto de Astrofísica, Facultad de Física, Pontificia Universidad Católica de Chile}$
$\newcommand{\lasa}{Liberal Arts and Science Academy, Austin, Texas 78724, USA}$
$\newcommand{\gemini}{Gemini Observatory/NSF’s NOIRLab, 670 N. A’ohoku Place, Hilo, HI, 96720, USA}$
$\newcommand{\umd}{Department of Astronomy, University of Maryland, College Park, College Park, MD}$
$\newcommand{\Bern}{Physikalisches Institut, University of Bern, Gesellschaftsstrasse 6, 3012 Bern, Switzerland}$
$\newcommand{\eberly}{\altaffiliation{Eberly Research Fellow}}$
$\newcommand{\torres}{\altaffiliation{Juan Carlos Torres Fellow}}$
$\newcommand{\sagan}{\altaffiliation{NASA Sagan Fellow}}$
$\newcommand{\bernoulli}{\altaffiliation{Bernoulli fellow}}$
$\newcommand{\gruber}{\altaffiliation{Gruber fellow}}$
$\newcommand{\kavli}{\altaffiliation{Kavli Fellow}}$
$\newcommand{\peg}{\altaffiliation{51 Pegasi b Fellow}}$
$\newcommand{\pappalardo}{\altaffiliation{Pappalardo Fellow}}$
$\newcommand{\hubble}{\altaffiliation{NASA Hubble Fellow}}$
$\newcommand{\nsf}{\altaffiliation{National Science Foundation Graduate Research Fellow}}$
$\newcommand{\maxplanck}{Max-Planck-Institut für Astronomie,Königstuhl 17, D-69117 Heidelberg, Germany}$
$\newcommand{\Sofia}{Department of Astronomy, Sofia University "St Kliment Ohridski", 5 James Bourchier Blvd, BG-1164 Sofia, Bulgaria}$
$\newcommand{\Heidelberg}{Landessternwarte, Zentrum für Astronomie der Universität Heidelberg, Königstuhl 12, D-69117 Heidelberg, Germany}$
$\newcommand{\Adolfo}{Facultad de Ingeniera y Ciencias, Universidad Adolfo Ibáñez, Av. Diagonal las Torres 2640, Peñalolén, Santiago, Chile}$
$\newcommand{\Millenium}{Millenium Institute of Astrophysics, Santiago, Chile}$</div>



<div id="title">

# Discovery and Characterization of an Eccentric, Warm Saturn Transiting the Solar Analog TOI-4994$\footnote{This paper includes data gathered with the 6.5 meter Magellan Telescopes located at Las Campanas Observatory, Chile.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.02769-b31b1b.svg)](https://arxiv.org/abs/2412.02769)<mark>Appeared on: 2024-12-05</mark> -  _17 pages, 10 figures. Accepted to AJ_

</div>
<div id="authors">

R. R. Martínez, et al. -- incl., <mark>J. Eberhardt</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We present the detection and characterization of TOI-4994b (TIC 277128619b), a warm Saturn-sized planet discovered by the NASA Transiting Exoplanet Survey Satellite (TESS). TOI-4994b transits a G-type star (V = 12.6 mag) with a mass, radius, and effective temperature of $M_{\star} =1.005^{+0.064}_{-0.061} M_{\odot}$ , $R_{\star} = 1.055^{+0.040}_{-0.037} R_{\odot}$ , and $T_{\rm eff} = 5640 \pm 110$ K. We obtained follow-up ground-based photometry from the Las Cumbres Observatory (LCO) and the Antarctic Search for Transiting ExoPlanets (ASTEP) telescopes, and we confirmed the planetary nature of TOI-4994b with multiple radial velocity observations from the PFS, CHIRON, HARPS, FEROS, and CORALIE instruments. From a global fit to the photometry and radial velocities, we determine that TOI-4994b is in a 21.5-day, eccentric orbit ( $e = 0.32 \pm 0.04$ ) and has a mass of $M_{P}= 0.280^{+0.037}_{-0.034} M_{J}$ , a radius of $R_{P}= 0.762^{+0.030}_{-0.027}R_{J}$ , and a Saturn-like bulk density of $\rho_{p} = 0.78^{+0.16}_{-0.14}$ $\rm g/cm^3$ . We find that TOI-4994 is a potentially viable candidate for follow-up stellar obliquity measurements. TOI-4994b joins the small sample of warm Saturn analogs and thus sheds light on our understanding of these rare and unique worlds.

</div>

<div id="div_fig1">

<img src="tmp_2412.02769/./Fig2.png" alt="Fig2" width="100%"/>

**Figure 2. -** Light curves of TOI-4994 from the LCO and ASTEP-Antarctica ground-based observatories.  (*fig:Ground_based*)

</div>
<div id="div_fig2">

<img src="tmp_2412.02769/./Fig4.png" alt="Fig4" width="100%"/>

**Figure 4. -** Phase-folded radial velocities from PFS (golden diamonds) and HARPS (purple stars and blue triangles) only, with residuals plotted below. (*fig:PFS_HARPS*)

</div>
<div id="div_fig3">

<img src="tmp_2412.02769/./Fig1.png" alt="Fig1" width="100%"/>

**Figure 1. -** TESS light curves of TOI-4994 at 30 (top), 10 (middle), and 2 (bottom) minute cadences, combining 6 different sectors. The red lines show the best-fit {\tt EXOFASTv2} models, phase-folded on the orbital period. (*fig:TESS_lcs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.02769"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Environmental Evidence for Overly Massive Black Holes in Low Mass Galaxies and a Black Hole - Halo Mass Relation at $z\sim5$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.02846-b31b1b.svg)](https://arxiv.org/abs/2412.02846)<mark>Appeared on: 2024-12-05</mark> -  _19 pages, 15 figures. Submitted to ApJ. Main results in Figures 12, 13, 14. Comments welcome_

</div>
<div id="authors">

J. Matthee, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** $\it$ JWST observations have unveiled faint active galactic nuclei (AGN) at high-redshift that provide insights on the formation of supermassive black holes (SMBHs) and their coevolution with galaxies. However, disentangling stellar from AGN light in these sources is challenging. Here, we use an empirical approach to infer the average stellar mass of 6 faint broad line (BL) H $\alpha$ emitters at $z=4-5$ with BH masses $\approx6  (4-15)\times10^6$ M $_{\odot}$ , with a method independent of their spectral energy distribution (SED). We use the deep $_ JWST_$ /NIRcam grism survey ALT to measure the over-densities around BL-H $\alpha$ emitters and around a spectroscopic reference sample of $\sim300$ galaxies. In our reference sample, we find that Mpc-scale over-density correlates with stellar mass, while pair counts are flat below $\approx50$ kpc due to satellites. Their large-scale environments suggest that BL-H $\alpha$ emitters are hosted by galaxies with stellar masses $\approx5\times10^7$ M $_{\odot}$ , $\approx40$ times lower than those inferred from galaxy-only SED fits. Adding measurements around more luminous $z\approx6$ AGNs, we find tentative correlations between line width, BH mass and the over-density, suggestive of a steep BH to halo mass relation. The main implications are (1) when BH masses are taken at face value, we confirm extremely high BH to stellar mass ratios of $\approx10$ \% , (2) the low stellar mass galaxies hosting growing SMBHs are in tension with typical hydrodynamical simulations, except those without feedback, (3) a 1 \% duty cycle implied by the host mass hints at super-Eddington accretion, which may imply over-estimated SMBH masses, (4) the masses are at odds with a high stellar density interpretation of the line broadening, (5) our results imply a diversity of galaxy masses, environments and SEDs among AGN samples, depending on their luminosity.

</div>

<div id="div_fig1">

<img src="tmp_2412.02846/./figs/MSTAR_MBH.png" alt="Fig10" width="100%"/>

**Figure 10. -** The relation between the stellar mass and SMBH mass for our BL-H$\alpha$ emitter sample (dark red hexagon, where stellar mass is indirectly inferred from their environments), a _ JWST_ literature compilation at $z\approx4-6$(blue points; where stellar mass is inferred from SED fitting, from  ([Carnall, McLure and Dunlop 2023](), [Maiolino, Scholtz and Curtis-Lake 2023](), [Harikane, Zhang and Nakajima 2023](), [Yue, Eilers and Simcoe 2024](), [Kokorev, Chisholm and Endsley 2024](), [Onoue, Ding and Silverman 2024]()) ). We highlight the luminous ALT-66543 (Labbe et al in prep) also with a red hexagon. For reference, we show the local relation measured by [Reines and Volonteri (2015)]() in black (showing the extrapolated range with a dashed line). The luminosity-bias corrected fit to _ JWST_ data from [Li, Silverman and Shen (2024)]() is shown in purple, purple contours show the expected observed distribution from this modeling work.  (*fig:Mstar_MBH*)

</div>
<div id="div_fig2">

<img src="tmp_2412.02846/./figs/MSTAR_MBH_sims.png" alt="Fig12" width="100%"/>

**Figure 12. -** The relation between the stellar mass and supermassive black hole mass at $z\approx5$, comparing measurements at $z\approx5$(dark red and blue points from data as in Fig. $\ref{fig:Mstar_MBH}$) with the 100 cMpc box EAGLE ( ([Schaye, Crain and Bower 2015](), [Crain, Schaye and Bower 2015]()) ) and Illustris-TNG ( ([Weinberger, Springel and Hernquist 2017](), [Pillepich, Springel and Nelson 2018]()) ) hydrodynamical simulations (orange and green, respectively). Red shows an EAGLE model without stellar feedback (ONLYAGN; with a 50 cMpc box size). Data-points show individual galaxies. Solid lines show the running median. Dashed lines show the running 95th percentile (where we note the low and high mass ends are impacted by low number statistics). The seed SMBH masses are $\approx10^5$ M$_{\odot}$ and $\approx10^6$ M$_{\odot}$ in EAGLE and Illustris-TNG, respectively. (*fig:Mstar_MBH_sims*)

</div>
<div id="div_fig3">

<img src="tmp_2412.02846/./figs/LHA_VFWHM.png" alt="Fig1" width="100%"/>

**Figure 1. -** Compilation of H$\alpha$ broad lines measured with _ JWST_ at $z=4-6$ from NIRCam Grism (red hexagons this paper; green squares EIGER \& FRESCO and ASPIRE in pink) and NIRspec (points, purple JADES, RUBIES and CEERS in blue and UNCOVER in orange) spectroscopy. We highlight how the H$\alpha$ luminosity and width change with SMBH mass and Eddington ratio under the commonly adopted calibrations used. Three of the ALT sources are also in the UNCOVER sample.  (*fig:lha_fwhm*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.02846"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\prelim}[1]{\colorbox{white!70!cyan}{\parbox{0.85\columnwidth}{\texttt{PRELIM: #1}}}}$
$\newcommand{\eureka}[1]{\texttt{Eureka!}{#1}}$
$\newcommand{\tiberius}[1]{\texttt{Tiberius}{#1}}$
$\newcommand{\exotic}[1]{\texttt{ExoTIC-JEDI}{#1}}$</div>



<div id="title">

# A dark, bare rock for TOI-1685 b from a JWST NIRSpec G395H phase curve

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.03411-b31b1b.svg)](https://arxiv.org/abs/2412.03411)<mark>Appeared on: 2024-12-05</mark> -  _26 pages, 19 figures, 8 tables. Submitted to AAS Journals_

</div>
<div id="authors">

R. Luque, et al. -- incl., <mark>E.-M. Ahrer</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ We report JWST NIRSpec/G395H observations of TOI-1685 b, a hot rocky super-Earth orbiting an M2.5V star, during a full orbit. We obtain transmission and emission spectra of the planet and characterize the properties of the phase curve, including its amplitude and offset. The transmission spectrum rules out clear $H_2$ -dominated atmospheres, while secondary atmospheres (made of water, methane, or carbon dioxide) cannot be statistically distinguished from a flat line. The emission spectrum is featureless and consistent with a blackbody-like brightness temperature, helping rule out thick atmospheres with high mean molecular weight. Collecting all evidence, the properties of TOI-1685 b are consistent with a blackbody with no heat redistribution and a low albedo, with a dayside brightness temperature $0.98\pm0.07$ times that of a perfect blackbody in the NIRSpec NRS2 wavelength range (3.823--5.172 µm). Our results add to the growing number of seemingly airless M-star rocky planets, thus constraining the location of the `Cosmic Shoreline'.Three independent data reductions have been carried out, all showing a high-amplitude correlated noise component in the white and spectroscopic light curves. The correlated noise properties are different between the NRS1 and NRS2 detectors -- importantly the timescales of the strongest components (4.5 hours and 2.5 hours, respectively) -- suggesting the noise is from instrumental rather than astrophysical origins. We encourage the community to look into the systematics of NIRSpec for long time-series observations.

</div>

<div id="div_fig1">

<img src="tmp_2412.03411/./figures/paper_whitepc.png" alt="Fig9" width="100%"/>

**Figure 9. -** The white-light phase curve of TOI-1685 b from 2.844-3.715 µm (NRS1, left column) and 3.823-5.172 µm (NRS2, right column) from JWST. Top: data from the $\eureka$ reduction (purple) and planet flux model fit to the data (gold). Bottom: residuals of the fit. Circles show binned data every 25 integrations to improve visualization.  (*fig:phasecurve*)

</div>
<div id="div_fig2">

<img src="tmp_2412.03411/./figures/nrs1_ls.png" alt="Fig7" width="100%"/>

**Figure 7. -** Results of running the residuals of the white-light and spectroscopic light curves through a Lomb-Scargle periodogram. (a) A gallery of the residuals from the \texttt{Eureka!} pipeline for each spectroscopic channel. (b) The periodogram results. The color of the lines between panels a and b are for the same white-light or spectroscopic channel. The residuals show uncorrelated noise at periods of $< 0.5$ hours. (*fig:nrs1_ls*)

</div>
<div id="div_fig3">

<img src="tmp_2412.03411/./figures/paper_emission.png" alt="Fig12" width="100%"/>

**Figure 12. -** Emission spectrum of TOI-1685 b using NIRSpec/G395H. Different data reductions follow the same color convention as Fig. \ref{fig:transmission_all}. Different symbols show different parts of the time series used in the fit as described in Sect. \ref{subsec:specchan}. Black circles represent the final emission spectrum from the prayer-bead analysis carried out in Sect. \ref{subsubsec:prayerbead}. The different datasets are offset slightly in wavelength to improve visualization.  (*fig:emission_all*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.03411"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

268  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
